In [1]:
# 通过意见表的意见，遍历文档找到仅出现一次的case，生成一个mark csv，即标志为科研通过code自动替换的。方便后面进行人工审核。
import csv
import docx
import xlrd
from tqdm import tqdm
import utils

In [2]:
# config
xlsx_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第四次会议/ALL.xlsx'

# in_path = '/home/gray/desktop/STANDARD/test2.docx' # example
in_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第四次会议/2.6.docx'

out_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第四次会议/out.docx'

doc = docx.Document(in_path)

In [27]:
# init replace dicts
replace = {}
replace_table = {}

workbook = xlrd.open_workbook(filename=xlsx_path)
worksheet = workbook.sheet_by_name(workbook.sheet_names()[0])

In [28]:
row = worksheet.row_values(0)
for i in range(11):
    print(i, row[i])

0 序号
1 标准章条编号
2 修改内容
3 
4 提出单位
5 处理意见及理由
6 备注
7 程序可处理
8 
9 
10 global index


In [29]:
for i in tqdm(range(3, worksheet.nrows)):
    row = worksheet.row_values(i)
#     sec_num = utils.get_sec_num(str(row[1]))
    sec_num = utils.get_sec_num_new(str(row[1]),from_head=True)
    
    pre_result = row[7]  # 0 -> deny, 1-> yes
    discuss = row[8]  # discuss flag
    before = str(row[2])
    after = str(row[3])
    global_index = row[10]
    
    if before == '':  # empty line
        continue
        
#     global_index = row[0]
    
    if discuss != 1 and pre_result != 0:  # 即 没有需要讨论 且 没有被拒绝
        if sec_num is not None:  # 
            # tmp solution: self-increment for a new added-in section
            tmp_s = [int(x) for x in sec_num.split('.')]
            tmp_s[0] += 1

            sec_num = ".".join([str(x) for x in tmp_s])
            
            if sec_num not in replace.keys():
                replace[sec_num] = []
            replace[sec_num].append((before,after,global_index))  # sec_num : (before, after, index)
        else:  # for table
            replace_table[before] = (after,global_index)  # before : (after, index)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 473/473 [00:00<00:00, 251127.32it/s]


In [30]:
# for k,v in replace.items():
#     for vv in v:
#         if vv[0] == '使用外部存储或是本地与外部混合的存储的系统称为存储虚拟化。':
#             print(k,vv)
# #             kk = k
#             break

In [31]:
# stat info
len_replace = 0
for v in replace.values():
    len_replace += len(v)
print("len of replace in text",len_replace)
print("len of replace in table",len(replace_table))

len of replace in text 423
len of replace in table 19


In [32]:
# replace里存的是xlsx里的替换规则
# ！！！！！！不check sec了，做全文的！！！！！！
replace_global = []
for v in replace.values():
    replace_global += v

# print(len(replace_global))
# print(replace_global[:10])
found = {}
for para in tqdm(list(doc.paragraphs)):
        for a,b,index in replace_global:
            if a != '' and a in para.text:
                if a not in found.keys():
                    found[a] = [0,b,index]
                found[a][0] += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11965/11965 [00:48<00:00, 245.30it/s]


In [16]:
# testing example

# key = "块服务中定义的空StoragePool，并添加了“空间限制”，“空间限制确定”和ThinProvisionMetaDataSpace属性。"

# key in [i[0] for i in replace_global]

# for para in tqdm(list(doc.paragraphs)):
#     if key in para.text:
#         print(para.text)
#         break

True

In [79]:
# # !!!!!这个是做sec内部只出现了一次的。
# # 搜索段落 每个只关联自己section内部（即section内部只出现一次）
# found = {}  # 格式 "before : [show_counter, index]"
# # search in paragraphs
# # now_sec = ''
# for para in tqdm(list(doc.paragraphs)):
#     # update sec_num
    
# #     sec_num = utils.get_sec_num(para.text)
# #     if sec_num is not None:
# #         now_sec = sec_num
# #     if now_sec == '':
# #         continue
# #     if now_sec not in replace.keys():
# #         continue

#     for sec in replace.keys():
#         for a,b,index in replace[sec]:
#             if a != '' and a in para.text:
#                 if a not in found.keys():
#                     found[a] = [0,b,index]
#                 found[a][0] += 1

100%|████████████████████████████████████████████████████████████████████████| 12046/12046 [00:36<00:00, 332.45it/s]


In [33]:
# 搜索表格
found_table = {}  # 格式 "before : [counter, afer, index]"
for table_index in tqdm(range(len(list(doc.tables)))):
    table = doc.tables[table_index]
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                for a,b in replace_table.items():
                    if a != '' and a in para.text:
                        if a not in found_table.keys():
                            found_table[a] = [0]+list(b)
                        found_table[a][0] += 1
# print(found_table)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 723/723 [00:10<00:00, 66.29it/s]


In [35]:
# search有匹配的数量
print(len(found))
print(len(found_table))

278
10


In [38]:
# 找到只出现一次的
indexes = []
for k,v in found.items():
    if v[0] ==  1:
        indexes.append(v[2])

for k,v in found_table.items():
    if v[0] <= 1:
        indexes.append(v[2])
        
print(len(indexes))
print(sorted(indexes)[:10])

247
[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]


In [39]:
sorted(indexes)

[2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 26.0,
 27.0,
 28.0,
 30.0,
 31.0,
 34.0,
 35.0,
 37.0,
 38.0,
 40.0,
 43.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 51.0,
 52.0,
 54.0,
 55.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 65.0,
 66.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 87.0,
 88.0,
 98.0,
 100.0,
 112.0,
 113.0,
 114.0,
 118.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 135.0,
 136.0,
 137.0,
 138.0,
 139.0,
 141.0,
 143.0,
 147.0,
 151.0,
 152.0,
 153.0,
 155.0,
 157.0,
 158.0,
 159.0,
 160.0,
 162.0,
 163.0,
 164.0,
 166.0,
 167.0,
 169.0,
 170.0,
 171.0,
 174.0,
 175.0,
 176.0,
 181.0,
 184.0,
 186.0,
 188.0,
 191.0,
 193.0,
 194.0,
 195.0,
 196.0,
 197.0,
 198.0,
 199.0,
 200.0,
 201.0,
 202.0,
 203.0,
 204.0,
 205.0,
 206.0,
 207.0,
 208.0,
 209.0,
 210.0,
 211.0,
 212.0,
 213.0,
 214.0,
 215.0,
 218.0,
 221.0,
 2

In [40]:
indexes_sorted = sorted([int(i) for i in indexes])
data = []
for i in range(1,int(indexes_sorted[-1])+1):
    if i in indexes_sorted:
        data.append([1])
    else:
        data.append([])

In [41]:
# please merge this detect results to xlsx manually
with open('/home/gray/desktop/detect.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [18]:
# -------------------------------below is for testing------------------------------------------------

In [78]:
now_sec = ''
for para in tqdm(list(doc.paragraphs)):
    sec_num = utils.get_sec_num(para.text)
    if sec_num is not None:
        now_sec = sec_num
    if now_sec == '':
        continue
    if "插入到底板或是母板的系统应该使用主机硬件RAID。" in para.text:
        print(para.text)
#         break
    

100%|██████████████████████████████████████████████████████████████████████| 12046/12046 [00:00<00:00, 44714.16it/s]

阵列配置文件描述了独立冗余磁盘阵列(RAID)系统。这个配置文件描述的RAID系统是独立的，它使用本地磁盘来存储数据。使用外部存储或是本地与外部混合的存储的系统称为存储虚拟化。插入到底板或是母板的系统应该使用主机硬件RAID。该文件见第7部分主机元素。另外，为支持广域网存储管理，阵列配置文件定义了RAID系统配置服务代理（service agent，SA）,和向S-P2PN中的管理节点（directory agent, DA）注册和注销的操作。


In [22]:
now_sec

'4.1.2'

In [23]:
utils.get_sec_num("4.1.2 概述")

'4.1.2'